# MgO Analysis Manual Scans

We start with manual scans from MgO 1800 nm short pulses

In [ ]:
## necessary import statements
from HARP.scan_anal import Scan
from HARP.iris_functions import Iris
from HARP.image_processing import Image
import os
from scipy.optimize import curve_fit
from scipy.signal import find_peaks, savgol_filter, peak_widths
import numpy as np 
import matplotlib.pyplot as plt
from functions import Andor_calib
from functions import Adams_wedge_thickness, gaus
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

In [ ]:
#Iris calibration from 28th Jan at 1800 nm short pulse
""" 1800 nm scans"""
power = np.array([780 ,745 ,635 ,490 ,300 ,95 ,10 ,10.5 ,14 ,20 ,26 ,33 ,41 ,50 ,59 ,69 ,80 ,10
,6 ,3 ,1.2 ,0 ,470])*1e-3
iris_pos = np.array([45,30,15	,0,-15,-30,-40,-39,-38,-37,-36,-35,-34,-33,-32,-31,-30,-40,-41,-42,-43,-44,0])


iris_27 = Iris()
iris_27.specify_calib(iris_positions=iris_pos, powers=power)
iris_27.specify_params(w0_init=100e-6, f=0.75, wl=800e-9, M2=1, reprate=1000,pulse_duration=15e-15)


In [ ]:
data_folder = '/Volumes/qolslc/20120009 - Matthews' 
data_folder = os.path.join(data_folder,'2022-01-27')
os.listdir(data_folder)

# Trans 30000
Manual Scans but this is the data we have from the MgO

In [ ]:
folder = os.path.join(data_folder,'Rotation_Scan_MgO_30000trans_min22deg_coarse')


In [ ]:
Rot = Scan()
Rot.set_folder(folder)
Rot.set_verlim(0, 5000)
files = [i for i in os.listdir(folder)]
bg = []
exclude = bg+[]
files = [i for i in files if i not in exclude]
print(files)

In [ ]:
files[1].split('d')

In [ ]:
variables = [float(i.split('d')[0]) for i in files]
Rot.set_params(wedge=1060, MCPPos=30000,iris=-22)
Rot.set_eVlim((8,35))
stage = 'rotation'
variables.sort()

print(variables)

In [ ]:
Rot.populate_scan_manual(files, variables, stage=stage, function = Andor_calib)
Rot.add_calibration_stage('intensity', iris_27.get_intensity_TWcm2, 'iris')
Rot.substract_bg(byitself=True, bg_lim=[0,0,10,10])
Rot.sort_by_stage(stage)

In [ ]:
fig, ax = plt.subplots(2)
im, ax[0], ax2 = Rot.plot_average(ax[0])
ax[0].set_title('Average Trace')
ax[0].xaxis.tick_top()
ax[0].set_xlabel('Energy [eV]', va = 'top')
ax[0].set_ylabel('Divergence [a.u.]')
im, ax[1] = Rot.plot_scan_mean(ax[1], stage)
ax[1].set_xlabel('Rotation')
ax[1].set_ylabel('Energy eV')
plt.show()

In [ ]:

x, y, Z = Rot.return_scan_data('rotation')
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(20,10))
r,theta = np.meshgrid(y,np.radians(x))
vmin = 10000
Z[Z<vmin]=vmin
pc = ax.contourf(theta, r, np.log10(Z), levels=50, cmap='nipy_spectral')
ax.set_thetalim(np.radians(min(x)), np.radians(max(x)))
ax.set_rlim(8,22)
cbc = plt.colorbar(pc, ax=ax)



# Intensity Scans

## Making the scan

In [ ]:
folder = os.path.join(data_folder,'Iris_Scan_MgO_800_trans40000')
scan = Scan()
scan.set_folder(folder)
scan.set_verlim(0, 5000)
files = [i for i in os.listdir(folder)]
bg = []
exclude = bg+[]
files = [i for i in files if i not in exclude]
print(files)



In [ ]:
variables = [-1*float(i.split('d')[0].split('n')[1]) for i in files]
scan.set_params(wedge=1060, MCPPos=40000,iris=-22)
scan.set_eVlim((8,35))
stage = 'intensity'
variables.sort()

print(variables)

In [ ]:
scan.populate_scan_manual(files, variables, stage='iris', function = Andor_calib)
scan.add_calibration_stage('intensity', iris_27.get_intensity_TWcm2, 'iris')
scan.substract_bg(byitself=True, bg_lim=[0,0,10,10])
scan.sort_by_stage(stage)


In [ ]:
scan.substract_bg(byitself=True, bg_lim=[0,0,10,10])

## Seeing what it looks like

In [ ]:
fig, ax = plt.subplots(2)
im, ax[0], ax2 = scan.plot_average(ax[0])
ax[0].set_title('Average Trace')
ax[0].xaxis.tick_top()
ax[0].set_xlabel('Energy [eV]', va = 'top')
ax[0].set_ylabel('Divergence [a.u.]')
im, ax[1] = scan.plot_scan_mean(ax[1], stage)
ax[1].set_xlabel(stage)
ax[1].set_ylabel('Energy eV')
plt.show()

In [ ]:
df = scan.scan_data[scan.scan_data['iris']<=-25]

In [ ]:
x, y, Z = scan.return_scan_data('intensity',df)

In [ ]:
fig, ax = plt.subplots(figsize = (20,10))
vmin=5000
Z[Z<vmin]=vmin
cma = ax.contourf(y,x,np.log10(Z), levels=50, cmap='nipy_spectral')
cba = plt.colorbar(cma, ax=ax)


In [ ]:
tr = scan.scan_data[scan.scan_data['iris']==-24]['Data'].values[0]

In [ ]:
tr.data

In [ ]:
plt.imshow(tr.data, cmap='nipy_spectral', vmax=20)
plt.colorbar()